## Testing assemble circuitry

In [1]:
import sys
from pathlib import Path

# Make sure Python can see src/ when running notebook
PROJECT_ROOT = Path.cwd().parents[0]  # if notebook is in /notebooks
sys.path.append(str(PROJECT_ROOT / "src"))

from data.loading import load_raw_gas
from data.assemble import build_basic_dataset

df_raw_no2 = load_raw_gas("NO2")
df_cycles_no2 = build_basic_dataset("NO2")
df_raw_no2.head(), df_cycles_no2.shape


(                                  V         I  MFC_target  flow_target_error  \
 Time                                                                           
 2022-02-15 18:58:28.000000  5.00077  0.003818         0.0                0.0   
 2022-02-15 18:58:28.125621  5.10065  0.003890         0.0                0.0   
 2022-02-15 18:58:28.253114  5.20053  0.003966         0.0                0.0   
 2022-02-15 18:58:28.381277  5.30060  0.004051         0.0                0.0   
 2022-02-15 18:58:28.509052  5.40056  0.004132         0.0                0.0   
 
                             flow_carrier_error  
 Time                                            
 2022-02-15 18:58:28.000000           98.809523  
 2022-02-15 18:58:28.125621           98.809523  
 2022-02-15 18:58:28.253114           98.809523  
 2022-02-15 18:58:28.381277           27.281745  
 2022-02-15 18:58:28.509052           10.714285  ,
 (1199, 402))